### ALS on Transactions
Plus blend options for Ratings, Bookmarks

In [1]:
import dataset_utils as du
import pandas as pd
import matplotlib.pyplot as plt
import metric_utils as mu
import als_model as als
import json
%matplotlib inline

### Tune Parameters

In [3]:
tr = du.processTransactions(use_cashe=True)
tr_train, tr_val = du.trainvalTransactions(tr, border=0.95)
# these params perform best:
rating_params = {'bookmarked': 7, 'purchased': 11, 
                             'watched_movie': 8, 'watched_series': 9, 'watch_failed': 2}

# try this mix
# rating_params = {'bookmarked': 5, 'purchased': 10, 
#                              'watched_movie': 8, 'watched_series': 9, 'watch_failed': 1}
ds_train = du.makeTarget(tr_train, du.processCatalogue(), du.processBookmarks(), du.processRatings(), params = rating_params)
ds_val = du.makeTarget(tr_val, du.processCatalogue(), du.processBookmarks(), du.processRatings(), params = rating_params)
ds_val = ds_val[ds_val['tr_rating'] > rating_params['watch_failed']] 
best_model, best_score, _ = als.tuneALS(ds_train, ds_val, factors = [10, 20, 30])

### Train/Predict

In [7]:
test_users = du.processTestUsers()
rating_params = {'bookmarked': 5, 'purchased': 11, 
                             'watched_movie': 8, 'watched_series': 9, 'watch_failed': 2}
ds_train_all = du.makeTarget(tr, du.processCatalogue(), du.processBookmarks(), du.processRatings(), 
                             blend_rating=False,
                             params = rating_params)

final_model, _, t_result = als.tuneALS(ds_train_all, ds_val[:1], factors = [30], t_users=test_users)

Test users done.
Elements filtered out from validation
Sparsity: 0.002326
Iteration 1 of 1



Model 1 fitted.
Started predict/score ...
Predict done.
MNAP@20: 0.0
Best model updated


In [5]:
len(t_result)

49992

In [6]:
import pickle
def saveSubmit(name, model, results):
    with open('submits/{0}.json'.format(name), 'w') as f:
        json.dump(results, f)
    print('Saved results {0}.json'.format(name))

    with open('submits/model_{0}.pkl'.format(name), 'wb') as f:
        pickle.dump(model, f)
    print('Saved model model_{0}.pkl'.format(name))
    
saveSubmit('als_ensure', final_model, t_result)
    


Saved results als_ensure.json
Saved model model_als_ensure.pkl
